In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import toml
import pickle
plt.rcParams['figure.figsize'] = (15, 6.5)

In [ ]:
# load parameters 
standard_params = toml.load("../foragingmodel/config_file.toml")
params = standard_params.copy()

start_year = 2017
N = 5

# delete parameters not included in SA
del params['resolution_min']
del params['init_birds']
del params['w_mussel_foraging_efficiency']
del params['s_worm_foraging_efficiency']
del params['mussel_density']
del params['relative_density']

# list of parameter names
param_vars = list(params.keys())

# load results
results = np.loadtxt('../results/sensitivity_analysis/sensitivity_standardparams_25percent.txt'.format(start_year)) 



vars = ['start_num_w', 'start_num_s', 'final_num_w', 'final_num_s', 'final_mean_foraging_w', 'final_mean_foraging_s',
        'final_mean_weight_w', 'final_mean_weight_s', 'end_mean_weight_w', 'end_mean_weight_s', 'mean_sumsq_weight_w',
        'mean_sumsq_weight_s']

# create dataframe of results
df = pd.DataFrame(data=results, columns=vars) 

# calculate mortality for both groups in percentage
df['mortality_w'] = 100 * (df['start_num_w'] - df['final_num_w']) / df['start_num_w']
df['mortality_s'] = 100 * (df['start_num_s'] - df['final_num_s']) / df['start_num_s']

# calculate means/std per N rows
df_means = df.groupby(np.arange(len(df)) // N).mean()
df_std = df.groupby(np.arange(len(df)) // N).std()

# split dataframe in increase/decrease
df_inc_mean = df_means.iloc[:int(df_means.shape[0]/2)]
df_dec_mean = df_means.iloc[int(df_means.shape[0]/2):]
df_inc_std = df_std.iloc[:int(df_std.shape[0]/2)]
df_dec_std = df_std.iloc[int(df_means.shape[0]/2):]

# reset index
df_dec_mean.reset_index(inplace=True)
df_dec_std.reset_index(inplace=True)

# rows to remove
remove = []
remove.append(param_vars.index('start_weight'))
remove.append(param_vars.index('minimum_weight'))

# remove from df's
df_inc_mean.drop(df_inc_mean.index[remove], inplace=True)
df_inc_std.drop(df_inc_std.index[remove], inplace=True)
df_dec_mean.drop(df_dec_mean.index[remove], inplace=True)
df_dec_std.drop(df_dec_std.index[remove], inplace=True)

# also remove from param_vars
for index in sorted(remove, reverse=True):
    del param_vars[index]

# reset index
df_dec_mean.reset_index(inplace=True)
df_inc_mean.reset_index(inplace=True)
df_dec_std.reset_index(inplace=True)
df_inc_std.reset_index(inplace=True)

# add param_vars to dataframe, then we can order everything!


# Load standard model run data
pkl_file = open('C:/Users/Marleen/Documents/thesis project/oystercatcher-model/results/single_simulation_runs/{}_10000agents_standardparams.pkl'.format(start_year), 'rb')
standard_results = pickle.load(pkl_file)
pkl_file.close()

# Standard Results

In [ ]:
mean_weight_w = []
mean_weight_s = []
mortality_s = []
mortality_w = []
mean_foraging_w = []
mean_foraging_s = []

# get data from standard results
for model_run in standard_results:
    mean_weight_w.append(np.mean(model_run['mean_weight_w']))
    mean_weight_s.append(np.mean(model_run['mean_weight_s']))
    mortality_s.append(100 * (model_run['total_num_s'][0] - model_run['total_num_s'][-1])/model_run['total_num_s'][0])
    mortality_w.append(100 * (model_run['total_num_w'][0] - model_run['total_num_w'][-1])/model_run['total_num_w'][0])
    mean_foraging_w.append(np.mean(model_run['mean_foraging_w']))
    mean_foraging_s.append(np.mean(model_run['mean_foraging_s']))

standard_mean_weight_w = np.mean(mean_weight_w)
standard_std_weight_w = np.std(mean_weight_w)
standard_mean_weight_s = np.mean(mean_weight_s)
standard_std_weight_s = np.std(mean_weight_s)
standard_mortality_w = np.mean(mortality_w)
standard_std_mortality_w = np.std(mortality_w)
standard_mortality_s = np.mean(mortality_s)
standard_std_mortality_s = np.std(mortality_s)
standard_mean_foraging_w = np.mean(mean_foraging_w)
standard_std_foraging_w = np.std(mean_foraging_w)
standard_mean_foraging_s = np.mean(mean_foraging_s)
standard_std_foraging_s = np.std(mean_foraging_s)

# Parameter increase by 25%

In [ ]:
# mean foraging time
plt.figure(1)
parameter = 'final_mean_foraging_w'
x = df_inc_mean.index
y = df_inc_mean[parameter]
std = df_inc_std[parameter]
plt.xticks(x, param_vars, rotation='vertical', size=10, fontsize=12)
plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='red')
plt.axhline(standard_mean_foraging_w)

plt.figure(2)

# increase by 25%
plt.bar(x,y-standard_mean_foraging_w, bottom=standard_mean_foraging_w)
plt.axhline(standard_mean_foraging_w, color='magenta')
plt.xticks(x, param_vars, rotation='vertical', size=10, fontsize=12)

# decrease
y = df_dec_mean[parameter]
plt.bar(x,y-standard_mean_foraging_w, bottom=standard_mean_foraging_w)
plt.show()


# # mean foraging time
# parameter = 'final_mean_foraging_w'
# x = df_dec_mean.index
# y = df_dec_mean[parameter]
# std = df_inc_std[parameter]
# plt.xticks(x, param_vars, rotation='vertical')
# plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='red')

# parameter = 'final_mean_foraging_s'
# x = df_inc_mean.index
# y = df_inc_mean[parameter]
# std = df_inc_std[parameter]
# plt.xticks(x, param_vars, rotation='vertical')
# plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='blue')
# plt.legend()
# plt.show()

In [ ]:
# mean deviation from ref weight
parameter = 'mean_sumsq_weight_w'
x = df_inc_mean.index
y = df_inc_mean[parameter]
std = df_inc_std[parameter]
plt.xticks(x, param_vars, rotation='vertical')
plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='red')

parameter = 'mean_sumsq_weight_s'
x = df_inc_mean.index
y = df_inc_mean[parameter]
std = df_inc_std[parameter]
plt.xticks(x, param_vars, rotation='vertical')
plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='blue')
plt.legend()
plt.show()

In [ ]:
# mean weight
parameter = 'final_mean_weight_w'
x = df_inc_mean.index
y = df_inc_mean[parameter]
std = df_inc_std[parameter]
plt.xticks(x, param_vars, rotation='vertical')
plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='red')

parameter = 'final_mean_weight_s'
x = df_inc_mean.index
y = df_inc_mean[parameter]
std = df_inc_std[parameter]

plt.xticks(x, param_vars, rotation='vertical')
plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='blue')
plt.legend()
plt.show()

In [ ]:
# final_weight
parameter = 'end_mean_weight_w'
x = df_inc_mean.index
y = df_inc_mean[parameter]
std = df_inc_std[parameter]
plt.xticks(x, param_vars, rotation='vertical')
plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='red')

parameter = 'end_mean_weight_s'
x = df_inc_mean.index
y = df_inc_mean[parameter]
std = df_inc_std[parameter]
plt.xticks(x, param_vars, rotation='vertical')
plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='blue')
plt.legend()
plt.show()

In [ ]:
# mortality
parameter = 'mortality_w'
x = df_inc_mean.index
y = df_inc_mean[parameter]
std = df_inc_std[parameter]
plt.xticks(x, param_vars, rotation='vertical')
plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='red')

parameter = 'mortality_s'
x = df_inc_mean.index
y = df_inc_mean[parameter]
std = df_inc_std[parameter]
plt.xticks(x, param_vars, rotation='vertical')
plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='blue')
plt.legend()


# # mortality
# parameter = 'mortality_w'
# x = df_inc_mean.index
# y = df_dec_mean[parameter]
# std = df_dec_std[parameter]
# plt.xticks(x, param_vars, rotation='vertical')
# plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='green')

# parameter = 'mortality_s'
# x = df_inc_mean.index
# y = df_dec_mean[parameter]
# std = df_dec_std[parameter]
# plt.xticks(x, param_vars, rotation='vertical')
# plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='magenta')
# plt.legend()
plt.show()

# Parameter decrease by 25%

In [ ]:
# mean foraging time
parameter = 'final_mean_foraging_w'
x = df_dec_mean.index
y = df_dec_mean[parameter]
std = df_inc_std[parameter]
plt.xticks(x, param_vars, rotation='vertical')
plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='red')

parameter = 'final_mean_foraging_s'
x = df_dec_mean.index
y = df_dec_mean[parameter]
std = df_dec_std[parameter]
plt.xticks(x, param_vars, rotation='vertical')
plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='blue')
plt.legend()
plt.show()

In [ ]:
# mean deviation from ref weight
parameter = 'mean_sumsq_weight_w'
x = df_dec_mean.index
y = df_dec_mean[parameter]
std = df_dec_std[parameter]
plt.xticks(x, param_vars, rotation='vertical')
plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='red')

parameter = 'mean_sumsq_weight_s'
x = df_dec_mean.index
y = df_dec_mean[parameter]
std = df_dec_std[parameter]
plt.xticks(x, param_vars, rotation='vertical')
plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='blue')
plt.legend()
plt.show()

In [ ]:
# mean weight
parameter = 'final_mean_weight_w'
x = df_dec_mean.index
y = df_dec_mean[parameter]
std = df_dec_std[parameter]
plt.xticks(x, param_vars, rotation='vertical')
plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='red')

parameter = 'final_mean_weight_s'
x = df_dec_mean.index
y = df_dec_mean[parameter]
std = df_dec_std[parameter]
plt.xticks(x, param_vars, rotation='vertical')
plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='blue')
plt.legend()
plt.show()

In [ ]:
# final_weight
parameter = 'end_mean_weight_w'
x = df_dec_mean.index
y = df_dec_mean[parameter]
std = df_dec_std[parameter]
plt.xticks(x, param_vars, rotation='vertical')
plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='red')

parameter = 'end_mean_weight_s'
x = df_dec_mean.index
y = df_dec_mean[parameter]
std = df_dec_std[parameter]
plt.xticks(x, param_vars, rotation='vertical')
plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='blue')
plt.legend()
plt.show()

In [ ]:
# mortality
parameter = 'mortality_w'
x = df_dec_mean.index
y = df_dec_mean[parameter]
std = df_dec_std[parameter]
plt.xticks(x, param_vars, rotation='vertical')
plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='red')

parameter = 'mortality_s'
x = df_dec_mean.index
y = df_dec_mean[parameter]
std = df_dec_std[parameter]
plt.xticks(x, param_vars, rotation='vertical')
plt.errorbar(x, y, std, linestyle='None',  markersize= 4, capsize=4, marker='o', color='blue')
plt.legend()
plt.show()

# Bar charts

In [ ]:
# # mortality
# parameter = 'mortality_s'

# # standard params
# standard_value = standard_mortality_s
# std_standard_value = standard_std_mortality_s

# # data
# x = df_inc_mean.index
# y = df_inc_mean[parameter]
# std = df_inc_std[parameter]

# # c = Counter(y).items()
# # print(c)
# # c = sorted(c, reverse=True)
# # labels, values = zip(*c)
# # indexes = np.arange(len(labels))
# # print(c)
# # plt.figure(1)
# print(std_standard_value)
# # increase by 25%
# plt.bar(x,y-standard_value, bottom=standard_value, yerr=std, capsize=4, label='25% increase')
# plt.axhline(standard_value, color='grey', linewidth=0.5)
# plt.axhline(standard_value + std_standard_value, color='grey', linewidth=0.5)
# plt.axhline(standard_value - std_standard_value, color='grey', linewidth=0.5)
# plt.xticks(x, param_vars, rotation='vertical', size=10, fontsize=10)

# # decrease
# y = df_dec_mean[parameter]
# std = df_dec_std[parameter]
# plt.bar(x,y-standard_value, bottom=standard_value, yerr=std, capsize=4, label='25% decrease')
# plt.legend()
# plt.show()

# mortality
plt.figure(1)
parameter = 'mortality_w'

# standard params
standard_value = standard_mortality_w
std_standard_value = standard_mortality_w

# decrease
y = df_dec_mean[parameter]
std = df_dec_std[parameter]
plt.bar(x,y-standard_value, bottom=standard_value, yerr=std, capsize=4, label='25 % decrease', alpha=0.5, color='blue')

# increase by 25%
x = df_inc_mean.index
y = df_inc_mean[parameter]
std = df_inc_std[parameter]
plt.bar(x,y-standard_value, bottom=standard_value, yerr=std, capsize=4, label='25% increase', alpha=0.6,color='red')
plt.axhline(standard_value, color='grey', linewidth=0.5)
plt.axhline(standard_value + std_standard_value, color='grey', linewidth=0.5)
plt.axhline(standard_value - std_standard_value, color='grey', linewidth=0.5)
plt.xticks(x, param_vars, rotation='vertical', size=10, fontsize=10)
plt.legend()
plt.show()

plt.figure(2)
parameter = 'mortality_s'

# standard params
standard_value = standard_mortality_s
std_standard_value = standard_mortality_s

# decrease
y = df_dec_mean[parameter]
std = df_dec_std[parameter]
plt.bar(x,y-standard_value, bottom=standard_value, yerr=std, capsize=4, label='25 % decrease', alpha=0.5, color='blue')

# increase by 25%
x = df_inc_mean.index
y = df_inc_mean[parameter]
std = df_inc_std[parameter]
plt.bar(x,y-standard_value, bottom=standard_value, yerr=std, capsize=4, label='25% increase', alpha=0.6,color='red')
plt.axhline(standard_value, color='grey', linewidth=0.5)
plt.axhline(standard_value + std_standard_value, color='grey', linewidth=0.5)
plt.axhline(standard_value - std_standard_value, color='grey', linewidth=0.5)
plt.xticks(x, param_vars, rotation='vertical', size=10, fontsize=10)
plt.legend()
plt.show()

In [ ]:
# # mean foraging time
# parameter = 'final_mean_foraging_w'
# standard_value = standard_mean_foraging_w
# x = df_inc_mean.index
# y = df_inc_mean[parameter]
# std = df_inc_std[parameter]

# plt.figure(1)

# # increase by 25%
# plt.bar(x,y-standard_value, bottom=standard_value, label='increase')
# plt.axhline(standard_value, color='magenta')
# plt.xticks(x, param_vars, rotation='vertical', size=10, fontsize=12)

# # decrease
# y = df_dec_mean[parameter]
# plt.bar(x,y-standard_value, bottom=standard_value, label='decrease')
# plt.legend()
# plt.show()

# mean foraging time
parameter = 'final_mean_foraging_w'

# standard params
standard_value = standard_mean_foraging_w
std_standard_value = standard_std_foraging_w

# decrease
plt.figure(1)
y = df_dec_mean[parameter]
std = df_dec_std[parameter]
plt.bar(x,y-standard_value, bottom=standard_value, yerr=std, capsize=4, label='25 % decrease', alpha=0.5, color='blue')

# increase by 25%
x = df_inc_mean.index
y = df_inc_mean[parameter]
std = df_inc_std[parameter]
plt.bar(x,y-standard_value, bottom=standard_value, yerr=std, capsize=4, label='25% increase', alpha=0.6,color='red')
plt.axhline(standard_value, color='grey', linewidth=0.5)
plt.axhline(standard_value + std_standard_value, color='grey', linewidth=0.5)
plt.axhline(standard_value - std_standard_value, color='grey', linewidth=0.5)
plt.xticks(x, param_vars, rotation='vertical', size=10, fontsize=10)
plt.legend()
plt.show()

# mean foraging time
parameter = 'final_mean_foraging_s'

# standard params
standard_value = standard_mean_foraging_s
std_standard_value = standard_std_foraging_s

# decrease
plt.figure(2)
y = df_dec_mean[parameter]
std = df_dec_std[parameter]
plt.bar(x,y-standard_value, bottom=standard_value, yerr=std, capsize=4, label='25 % decrease', alpha=0.5, color='blue')

# increase by 25%
x = df_inc_mean.index
y = df_inc_mean[parameter]
std = df_inc_std[parameter]
plt.bar(x,y-standard_value, bottom=standard_value, yerr=std, capsize=4, label='25% increase', alpha=0.6,color='red')
plt.axhline(standard_value, color='grey', linewidth=0.5)
plt.axhline(standard_value + std_standard_value, color='grey', linewidth=0.5)
plt.axhline(standard_value - std_standard_value, color='grey', linewidth=0.5)
plt.xticks(x, param_vars, rotation='vertical', size=10, fontsize=10)
plt.legend()
plt.show()

In [ ]:
# # mean foraging time
# parameter = 'final_mean_weight_s'
# standard_value = standard_mean_weight_s
# x = df_inc_mean.index
# y = df_inc_mean[parameter]
# std = df_inc_std[parameter]

# plt.figure(1)

# # increase by 25%
# plt.bar(x,y-standard_value, bottom=standard_value)
# plt.axhline(standard_value, color='magenta')
# plt.xticks(x, param_vars, rotation='vertical', size=10, fontsize=12)

# # decrease
# y = df_dec_mean[parameter]
# plt.bar(x,y-standard_value, bottom=standard_value)
# plt.show()


# parameter = 'final_mean_weight_w'

# # standard params
# standard_value = standard_mean_weight_s
# std_standard_value = standard_std_weight_s

# # decrease
# y = df_dec_mean[parameter]
# std = df_dec_std[parameter]
# plt.bar(x,y-standard_value, bottom=standard_value, yerr=std, capsize=4, label='25 % decrease', alpha=0.5, color='blue')

# # data
# x = df_inc_mean.index
# y = df_inc_mean[parameter]
# std = df_inc_std[parameter]

# # increase by 25%
# plt.bar(x,y-standard_value, bottom=standard_value, yerr=std, capsize=4, label='25% increase', alpha=0.6,color='red')
# plt.axhline(standard_value, color='grey', linewidth=0.5)
# plt.axhline(standard_value + std_standard_value, color='grey', linewidth=0.5)
# plt.axhline(standard_value - std_standard_value, color='grey', linewidth=0.5)
# plt.xticks(x, param_vars, rotation='vertical', size=10, fontsize=10)


# plt.legend()
# plt.show()

# mortality
plt.figure(1)
parameter = 'final_mean_weight_w'

# standard params
standard_value = standard_mean_weight_w
std_standard_value = standard_std_weight_w

# decrease
y = df_dec_mean[parameter]
std = df_dec_std[parameter]
plt.bar(x,y-standard_value, bottom=standard_value, yerr=std, capsize=4, label='25 % decrease', alpha=0.5, color='blue')

# increase by 25%
x = df_inc_mean.index
y = df_inc_mean[parameter]
std = df_inc_std[parameter]
plt.bar(x,y-standard_value, bottom=standard_value, yerr=std, capsize=4, label='25% increase', alpha=0.6,color='red')
plt.axhline(standard_value, color='grey', linewidth=0.5)
plt.axhline(standard_value + std_standard_value, color='grey', linewidth=0.5)
plt.axhline(standard_value - std_standard_value, color='grey', linewidth=0.5)
plt.xticks(x, param_vars, rotation='vertical', size=10, fontsize=10)
plt.legend()
plt.show()

plt.figure(2)
parameter = 'final_mean_weight_s'

# standard params
standard_value = standard_mean_weight_s
std_standard_value = standard_std_weight_s

# decrease
y = df_dec_mean[parameter]
std = df_dec_std[parameter]
plt.bar(x,y-standard_value, bottom=standard_value, yerr=std, capsize=4, label='25 % decrease', alpha=0.5, color='blue')

# increase by 25%
x = df_inc_mean.index
y = df_inc_mean[parameter]
std = df_inc_std[parameter]
plt.bar(x,y-standard_value, bottom=standard_value, yerr=std, capsize=4, label='25% increase', alpha=0.6,color='red')
plt.axhline(standard_value, color='grey', linewidth=0.5)
plt.axhline(standard_value + std_standard_value, color='grey', linewidth=0.5)
plt.axhline(standard_value - std_standard_value, color='grey', linewidth=0.5)
plt.xticks(x, param_vars, rotation='vertical', size=10, fontsize=10)
plt.legend()
plt.show()

In [ ]:
([a for a in range(3)])

# Plot function

In [ ]:
def create_sa_plot(parameter, standard_mean, standard_std, ylabel, title, ylim):
    
    # standard params
    standard_value = standard_mean
    std_standard_value = standard_std

    # decrease
    x = df_inc_mean.index
    y = df_dec_mean[parameter]
    std = df_dec_std[parameter]
    plt.grid(zorder=0)    
    plt.bar(x,y-standard_value, bottom=standard_value, yerr=std, capsize=4, label='25 % decrease', alpha=0.5, color='blue', zorder=3)

    # increase by 25%
    
    y = df_inc_mean[parameter]
    std = df_inc_std[parameter]
    plt.bar(x,y-standard_value, bottom=standard_value, yerr=std, capsize=4, label='25% increase', alpha=0.6,color='red', zorder=2)
    plt.axhline(standard_value, color='grey', linewidth=0.5)
    plt.axhline(standard_value + std_standard_value, color='grey', linewidth=0.5)
    plt.axhline(standard_value - std_standard_value, color='grey', linewidth=0.5)
    plt.xticks(x, param_vars, rotation='vertical', size=10, fontsize=12)
    plt.ylim(ylim[0], ylim[1])
    plt.xlim(0-1,len(x))
    
    plt.ylabel(ylabel)
    plt.legend()
    plt.title(title, fontsize=15)
    plt.tight_layout()
    
    plt.savefig('../figures/sensitivity_analysis/' + parameter +'.png')
    plt.show()

In [ ]:
# mortality
parameter = 'mortality_s'
standard_value = standard_mortality_s
std_standard_value = standard_mortality_s
create_sa_plot(parameter, standard_value, std_standard_value, 'Mortality (%)', 'Shellfish specialists', [-2, 100])
parameter = 'mortality_w'
standard_value = standard_mortality_w
std_standard_value = standard_mortality_w
create_sa_plot(parameter, standard_value, std_standard_value, 'Mortality (%)', 'Worm specialists', [-2, 100])


# foraging time
parameter = 'final_mean_foraging_s'
standard_value = standard_mean_foraging_s
std_standard_value = standard_std_foraging_s
create_sa_plot(parameter, standard_value, std_standard_value, 'Mean foraging time per tidal (h)', 'Shellfish specialists', [3, 8])
parameter = 'final_mean_foraging_w'
standard_value = standard_mean_foraging_w
std_standard_value = standard_std_foraging_w
create_sa_plot(parameter, standard_value, std_standard_value, 'Mean foraging time per tidal (h)', 'Worm specialists', [3, 8])

# foraging time
parameter = 'final_mean_weight_s'
standard_value = standard_mean_weight_s
std_standard_value = standard_std_weight_s
create_sa_plot(parameter, standard_value, std_standard_value, 'Mean weight (g)', 'Shellfish specialists', [550, 600])
parameter = 'final_mean_weight_w'
standard_value = standard_mean_weight_w
std_standard_value = standard_std_weight_w
create_sa_plot(parameter, standard_value, std_standard_value, 'Mean weight (g)', 'Worm specialists', [550, 600])